In [ ]:
import numpy as np
import jax.numpy as jnp

import json as js

import math

import sys
sys.path.append(r"./src")
from src.impl import sqp
from src.impl import myplot

In [1]:
import numpy as np
import jax
import jax.numpy as jnp
from jax import jacfwd, jacrev, hessian

import json as js

import math


In [3]:
def f(x):
    return [-sum(x**2),-sum(x**3)]
def f2(x):
    return sum(x**2)
fh=hessian(f)
a=jnp.array([1.,2.,3.,4.])
g=lambda x:sum(fh(x))
sum(fh(jnp.array([1.,2.,3.,4.])))
g(jnp.array([1.,2.,3.,4.]))
fj=jacfwd(f)
A=np.vstack(fj(a))
np.shape(A)
np.hstack([a,1.2])
np.shape(f(a))[0]

2

In [20]:
def IPM(f,cons,A,b,x0,lmd=0.1,gamma=2,epsf=1e-4,eps=1e-4,beta=0.5,tau=0.01):
    cons_num=np.shape(cons(x0))[0]
    A_num=np.shape(A)[0]
    x_num=np.shape(x0)[0]
    y=np.hstack((x0,lmd*np.ones(cons_num),lmd*np.ones(A_num)))
    
    # compute hessian and jacobi
    f_hess=hessian(f)
    f_jac=jacfwd(f)
    cons_jac=jacfwd(cons)
    cons_jac_mat=lambda x:np.vstack(cons_jac(x))
    cons_hess=hessian(cons)
    if cons_num>1:
        cons_hess_sum=lambda x,lmd:np.average(cons_hess(x),axis=0,weights=lmd)*(sum(lmd)/cons_num)
    else:
        cons_hess_sum=lambda x,lmd:lmd*cons_hess(x)[0]
    
    t=1
    itr=1
    #while 1:
    while 1:
        # variables
        x=y[0:x_num]
        lmd=y[x_num:x_num+cons_num]
        v=y[x_num+cons_num:]

        # assemble matrix
        r_dual=f_jac(x)+np.matmul(lmd,cons_jac_mat(x))+np.matmul(v,A)
        r_cent=-lmd*cons(x)-1/t
        r_pri=np.matmul(A,np.transpose(x))-b
        
        RH=-np.hstack((r_dual,r_cent,r_pri))
        RH_norm=np.linalg.norm(RH)
        #print(RH)
        
        lh11=f_hess(x)+cons_hess_sum(x,lmd)
        lh12=np.transpose(cons_jac_mat(x))
        lh13=np.transpose(A)
        lh21=-np.transpose(lh12*lmd)
        lh22=-np.diag(cons(x))
        
        LH1=np.hstack((lh11,lh12,lh13))
        LH2=np.hstack((lh21,lh22,np.zeros((cons_num,A_num))))
        LH3=np.hstack((A,np.zeros((A_num,cons_num+A_num))))
        
        LH=np.vstack((LH1,LH2,LH3))
        dy=np.linalg.solve(LH,RH)
        # line search
        dlmd=dy[x_num:x_num+cons_num]
        where_dlmd=np.where(dlmd>=0)
        divide=-np.delete(lmd,where_dlmd)/np.delete(dlmd,where_dlmd)
        alf_max=1
        if np.size(divide):
            alf_max=min(1,min(divide))
        alf=alf_max*0.99
        while 1:
            if (sum(np.array(cons(y+alf*dy))>=0)==0):
                break
            alf=alf*beta
        while 1:
            y_tmp=y+alf*dy
            
            x_tmp=y_tmp[0:x_num]
            lmd_tmp=y_tmp[x_num:x_num+cons_num]
            v_tmp=y_tmp[x_num+cons_num:]
            
            wr_dual=f_jac(x_tmp)+np.matmul(lmd_tmp,cons_jac_mat(x_tmp))+np.matmul(v_tmp,A)
            wr_cent=-lmd_tmp*cons(x_tmp)-1/t
            wr_pri=np.matmul(A,np.transpose(x_tmp))-b     
            wRH=-np.hstack((wr_dual,wr_cent,wr_pri))
            wRH_norm=np.linalg.norm(wRH)
            if(wRH_norm<=(1-tau*alf)*RH_norm):
                break
            alf=alf*beta
        print(alf)
        y=y+alf*dy
        
        # variables
        x=y[0:x_num]
        lmd=y[x_num:x_num+cons_num]
        v=y[x_num+cons_num:]
        
        eta=-np.dot(cons(x),lmd)
        # assemble matrix
        r_dual=f_jac(x)+np.matmul(lmd,cons_jac_mat(x))+np.matmul(v,A)
        r_pri=np.matmul(A,np.transpose(x))-b
        
        norm_dual=np.linalg.norm(r_dual)
        norm_pri=np.linalg.norm(r_pri)
        
        if((norm_dual<=epsf)&(norm_pri<=epsf)&(eta<=eps)):
            break
        print("iter: ",itr,"x = ",x,"t= ",t)
        itr=itr+1
        t=gamma*cons_num/eta
    return x

In [21]:
def rosen(x):
    """The Rosenbrock function"""
    return x[0]**2+x[1]**2

def cons(x):
    return [x[0]**2+x[1]**2-1]
A=np.array([[1.,-1.]])
b=np.array([0.])
x0=np.array([0.5, 0.5])
x=IPM(rosen,cons,A,b,x0)
print(x)
#print(np.shape(x0))
#cons_hess=hessian(cons)
#lmd=np.array([1.,2.,3.])
#print(cons_hess(x0))
#print(np.average(cons_hess(x0),axis=0,weights=lmd)*(sum(lmd)/3))

0.495
iter:  1 x =  [-0.07115384 -0.07115384] t=  1
0.99
iter:  2 x =  [-0.01699195 -0.01699195] t=  2.4881316889219387
0.99
iter:  3 x =  [-0.00256677 -0.00256677] t=  5.035195315098147
0.99
iter:  4 x =  [-0.00023771 -0.00023771] t=  9.984453325674393
0.99
iter:  5 x =  [-1.31863738e-05 -1.31863738e-05] t=  19.77187266747699
0.99
iter:  6 x =  [-4.49088726e-07 -4.49088726e-07] t=  39.1522352446756
0.99
iter:  7 x =  [-1.00812657e-08 -1.00812657e-08] t=  77.52917877853969
0.99
iter:  8 x =  [-1.64985682e-10 -1.64985682e-10] t=  153.52312629431586
0.99
iter:  9 x =  [-2.18362654e-12 -2.18362654e-12] t=  304.00619068181373
0.99
iter:  10 x =  [-2.54153569e-14 -2.54153569e-14] t=  601.9924567956708
0.99
iter:  11 x =  [-2.75224326e-16 -2.75224326e-16] t=  1192.0642708825164
0.99
iter:  12 x =  [-2.8675529e-18 -2.8675529e-18] t=  2360.5233086782505
0.99
iter:  13 x =  [-2.92825477e-20 -2.92825477e-20] t=  4674.30358154109
0.99
iter:  14 x =  [-2.9595743e-22 -2.9595743e-22] t=  9256.046696

In [25]:
10/3

3.3333333333333335

In [31]:
asd=np.random.randint(0,2,size=(4,4))
print(asd)
np.matmul(asd,np.transpose(a))
np.transpose(np.transpose(asd)*a)
np.diag(a)
np.hstack((asd,np.zeros((4,4))))
np.linalg.norm(a,1)

[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 1]
 [0 0 0 1]]


10.0

In [34]:
(1>0)&(2>1)

True

In [47]:
b=a-2
(b<0)*b*a
min(1,2)

1

In [54]:
c=np.where(b==0)
np.delete(b,c)
b/a

DeviceArray([-1.        ,  0.        ,  0.33333334,  0.5       ], dtype=float32)

In [61]:
if np.sum(b<0,0)>0:
    print("nigger")

nigger


In [66]:
False==0

True